In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir, exists
from datetime import datetime
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

In [ ]:
combined_results_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/caron_and_montoye/combined_results_ANN_hip_2015/'
combined_data_files =  [f for f in listdir(combined_results_folder) if isfile(join(combined_results_folder, f))]
output_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/hip_minute_data/'
print(len(combined_data_files))
ActiPAL_output_folder = output_folder

combined_ids = [f.split('_')[0] for f in listdir(combined_results_folder) if isfile(join(combined_results_folder, f))]
combined_ids = list(set(combined_ids))
print(combined_ids)
print(len(combined_ids))

1197
['KOA63', 'KOA30', 'KOA24', 'KOA27', 'KOA31', 'OA70', 'KOA8', 'HOA8', 'OA63', 'OA30', 'HOA11', 'OA27', 'HOA7', 'OA61', 'HOA30', 'OA81', 'KOA2', 'OA14', 'HOA27', 'HOA14', 'KOA57', 'OA36', 'KOA74', 'OA75', 'HOA43', 'OA22', 'OA33', 'OA45', 'KOA61', 'OA64', 'OA10', 'KOA3', 'HOA35', 'KOA16', 'OA16', 'HOA46', 'KOA9', 'OA56', 'KOA25', 'OA44', 'OA26', 'OA74', 'HOA34', 'HOA20', 'HOA29', 'HOA17', 'KOA65', 'KOA20', 'KOA56', 'OA40', 'KOA22', 'OA43', 'OA38', 'OA66', 'OA76', 'HOA9', 'KOA71', 'KOA26', 'OA51', 'HOA16', 'OA12', 'OA71', 'KOA55', 'KOA47', 'OA37', 'KOA29', 'OA58', 'OA72', 'OA79', 'OA52', 'KOA49', 'KOA10', 'OA31', 'OA77', 'HOA32', 'OA53', 'KOA66', 'OA35', 'HOA37', 'OA24', 'HOA15', 'OA73', 'HOA39', 'HOA33', 'KOA5', 'OA25', 'KOA50', 'HOA41', 'OA78', 'OA62', 'HOA40', 'KOA12', 'KOA67', 'KOA4', 'HOA4', 'OA2', 'OA42', 'HOA26', 'OA41', 'KOA6', 'OA49', 'KOA11', 'KOA51', 'HOA1', 'KOA1', 'KOA18', 'OA21', 'OA23', 'HOA2', 'OA55', 'OA67', 'OA54', 'OA47', 'OA65', 'OA11', 'HOA19', 'KOA7', 'HOA24', '

### Generate statistical features

In [ ]:
def met_to_intensity_c1(row):
  ee = per_person_df['caron_1'][row.name]
  return 1 if ee <= 1.5 else 2 if ee < 3 else 3

def met_to_intensity_c2(row):
  ee = per_person_df['caron_2'][row.name]
  return 1 if ee <= 1.5 else 2 if ee < 3 else 3

def met_to_intensity_m1(row):
  ee = per_person_df['mont_1'][row.name]
  return 1 if ee <= 1.5 else 2 if ee < 3 else 3

def met_to_intensity_m2(row):
  ee = per_person_df['mont_2'][row.name]
  return 1 if ee <= 1.5 else 2 if ee < 3 else 3

def met_to_intensity_m3(row):
  ee = per_person_df['mont_3'][row.name]
  return 1 if ee <= 1.5 else 2 if ee < 3 else 3

def met_to_intensity_m4(row):
  ee = per_person_df['mont_4'][row.name]
  return 1 if ee <= 1.5 else 2 if ee < 3 else 3

def get_bmi_range(row):
  bmi = per_person_df['BMI'][row.name]
  if(18.5<=bmi<25):return 'normal'
  elif(25<=bmi<30):return 'overweight'
  elif(30<=bmi):return 'obese'
  else:'error'

In [ ]:
s = ['A']*10
print(s)

['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']


In [ ]:
all_df = pd.DataFrame()
for each_id in combined_ids:
    sex = ''
    bmi = ''
    complete_df = pd.DataFrame()
    # print(combined_data_files)
    for combined_file in combined_data_files:
        
        if (combined_file.split('_')[0] != each_id): continue
        print(combined_file)
        combined_data = pd.read_csv(combined_results_folder + combined_file)
        # print(len(combined_data))
        sex = combined_data['sex'][0]
        if(sex!='male' and sex!='female'):
          if(user_id in combined_ids):
            print(user_id)
            sex='female'
        bmi = combined_data['BMI'][0]
        if(str(bmi)=='nan'):
          bmi = 34.18751650116381
        
        temp_df = combined_data[['EE_caron_1', 'EE_caron_2', 'predicted_ee_montoye_2015_feature_set_3',
       'predicted_ee_montoye_2015_feature_set_2',
       'predicted_ee_montoye_2015_feature_set_1',
       'predicted_ee_montoye_2015_feature_set_4']]
        
        complete_df = pd.concat([complete_df, temp_df])
        
    
    print('all',len(complete_df))
    complete_df.reset_index(drop=True, inplace=True)
    complete_df_grouped = complete_df.groupby(complete_df.index // 2)
    print('all',len(complete_df_grouped))
    per_person_df = pd.DataFrame()
    per_person_df['Participant'] = [each_id]*len(complete_df_grouped)
    per_person_df['sex'] = [sex]*len(complete_df_grouped)
    per_person_df['BMI'] = [bmi]*len(complete_df_grouped)
    per_person_df['caron_1'] = complete_df_grouped['EE_caron_1'].mean()
    per_person_df['caron_2'] = complete_df_grouped['EE_caron_2'].mean()
    per_person_df['mont_1'] = complete_df_grouped['predicted_ee_montoye_2015_feature_set_1'].mean()
    per_person_df['mont_2'] = complete_df_grouped['predicted_ee_montoye_2015_feature_set_2'].mean()
    per_person_df['mont_3'] = complete_df_grouped['predicted_ee_montoye_2015_feature_set_3'].mean()
    per_person_df['mont_4'] = complete_df_grouped['predicted_ee_montoye_2015_feature_set_4'].mean()
    per_person_df['caron_1_cat'] = per_person_df.apply(met_to_intensity_c1,axis=1)
    per_person_df['caron_2_cat'] = per_person_df.apply(met_to_intensity_c2,axis=1)
    per_person_df['mont_1_cat'] = per_person_df.apply(met_to_intensity_m1,axis=1)
    per_person_df['mont_2_cat'] = per_person_df.apply(met_to_intensity_m2,axis=1)
    per_person_df['mont_3_cat'] = per_person_df.apply(met_to_intensity_m3,axis=1)
    per_person_df['mont_4_cat'] = per_person_df.apply(met_to_intensity_m4,axis=1)
    per_person_df['BMI_range'] = per_person_df.apply(get_bmi_range,axis=1)
    all_df = pd.concat([all_df, per_person_df])
    

# print(len(all_df))
out_file = output_folder+'2_per_minute_EE_means_hip_mont_15_caron.csv'
print(out_file)
all_df.to_csv(out_file)

  


KOA63_2019-10-14_15-00-00_to_2019-10-14_22-00-00_Epoch30_ANN_2015_hip_combined.csv
KOA63_2019-10-15_06-00-00_to_2019-10-15_22-00-00_Epoch30_ANN_2015_hip_combined.csv
KOA63_2019-10-16_06-00-00_to_2019-10-16_22-00-00_Epoch30_ANN_2015_hip_combined.csv
KOA63_2019-10-17_06-00-00_to_2019-10-17_22-00-00_Epoch30_ANN_2015_hip_combined.csv
KOA63_2019-10-18_06-00-00_to_2019-10-18_22-00-00_Epoch30_ANN_2015_hip_combined.csv
KOA63_2019-10-19_06-00-00_to_2019-10-19_22-00-00_Epoch30_ANN_2015_hip_combined.csv
KOA63_2019-10-21_06-00-00_to_2019-10-21_07-59-59_Epoch30_ANN_2015_hip_combined.csv
KOA63_2019-10-20_06-00-00_to_2019-10-20_22-00-00_Epoch30_ANN_2015_hip_combined.csv
all 12584
all 6292
KOA30_2019-07-29_17-00-00_to_2019-07-29_21-30-00_Epoch30_ANN_2015_hip_combined.csv
KOA30_2019-07-30_04-00-00_to_2019-07-30_21-30-00_Epoch30_ANN_2015_hip_combined.csv
KOA30_2019-07-31_04-00-00_to_2019-07-31_21-30-00_Epoch30_ANN_2015_hip_combined.csv
KOA30_2019-08-01_04-00-00_to_2019-08-01_21-30-00_Epoch30_ANN_2015_hi

In [ ]:
complete_df = pd.read_csv('/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/hip_minute_data/2_per_minute_EE_means_hip_mont_15_caron.csv')
# print(complete_df['BMI'].unique())
# print(len(complete_df[complete_df['sex']=='male']))
# print(len(complete_df[complete_df['sex']=='female']))
# print(len(complete_df[complete_df['BMI_range']=='overweight']))
# print(len(complete_df))
# test_d = complete_df.groupby(['sex','Participant'])
# print(test_d)
grouped_df_c1 = complete_df.groupby(['Participant','sex','BMI_range','caron_1_cat']).size().unstack(fill_value=0)
grouped_df_c1.to_csv(join(ActiPAL_output_folder, '3_per_minute_EE_means_hip_caron_1_grouped.csv'))

grouped_df_c2 = complete_df.groupby(['Participant','sex','BMI_range','caron_2_cat']).size().unstack(fill_value=0)
grouped_df_c2.to_csv(join(ActiPAL_output_folder, '3_per_minute_EE_means_hip_caron_2_grouped.csv'))

grouped_df_m1 = complete_df.groupby(['Participant','sex','BMI_range','mont_1_cat']).size().unstack(fill_value=0)
grouped_df_m1.to_csv(join(ActiPAL_output_folder, '3_per_minute_EE_means_hip_mont_1_grouped.csv'))

grouped_df_m2 = complete_df.groupby(['Participant','sex','BMI_range','mont_2_cat']).size().unstack(fill_value=0)
grouped_df_m2.to_csv(join(ActiPAL_output_folder, '3_per_minute_EE_means_hip_mont_2_grouped.csv'))

grouped_df_m3 = complete_df.groupby(['Participant','sex','BMI_range','mont_3_cat']).size().unstack(fill_value=0)
grouped_df_m3.to_csv(join(ActiPAL_output_folder, '3_per_minute_EE_means_hip_mont_3_grouped.csv'))

grouped_df_m4 = complete_df.groupby(['Participant','sex','BMI_range','mont_4_cat']).size().unstack(fill_value=0)
grouped_df_m4.to_csv(join(ActiPAL_output_folder, '3_per_minute_EE_means_hip_mont_4_grouped.csv'))

In [ ]:
def categorize(input_name,output_sheet_name):
  all_df = pd.read_csv(join(ActiPAL_output_folder,input_name))

  female = all_df['sex'] == 'female'
  male = all_df['sex'] == 'male'
  obese = all_df['BMI_range'] == 'obese'
  overweight = all_df['BMI_range'] == 'overweight'
  normal = all_df['BMI_range'] == 'normal'


  male_df = all_df[male]
  female_df = all_df[female]
  obese_df = all_df[obese]
  overweight_df = all_df[overweight]
  normal_df = all_df[normal]

  # print(['all,male,female',len(all_df),len(male_df),len(female_df)])
  # print(['all,male,female',all_df['1'].mean,len(male_df),len(female_df)])

  with pd.ExcelWriter(join(ActiPAL_output_folder,output_sheet_name)) as writer: 
      all_df.to_excel(writer, sheet_name='all')
      female_df.to_excel(writer, sheet_name='female')
      male_df.to_excel(writer, sheet_name='male')
      obese_df.to_excel(writer, sheet_name='obese')
      overweight_df.to_excel(writer, sheet_name='overweight')
      normal_df.to_excel(writer, sheet_name='normal')

In [ ]:
categorize('3_per_minute_EE_means_hip_caron_1_grouped.csv','4_hip_caron_1_minute_values.xlsx')
categorize('3_per_minute_EE_means_hip_caron_2_grouped.csv','4_hip_caron_2_minute_values.xlsx')
categorize('3_per_minute_EE_means_hip_mont_1_grouped.csv','4_hip_mont_1_minute_values.xlsx')
categorize('3_per_minute_EE_means_hip_mont_2_grouped.csv','4_hip_mont_2_minute_values.xlsx')
categorize('3_per_minute_EE_means_hip_mont_3_grouped.csv','4_hip_mont_3_minute_values.xlsx')
categorize('3_per_minute_EE_means_hip_mont_4_grouped.csv','4_hip_mont_4_minute_values.xlsx')
